In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# We will define some reading parametes and create a bert based regression model with concatenation of the different inputs

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import os
import re
from sklearn.model_selection import train_test_split
import spacy
nlp= spacy.load('en_core_web_sm')


from transformers import AdamW
from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import AutoConfig
from transformers import get_cosine_schedule_with_warmup
from transformers import BertTokenizer, BertModel, BertConfig,BertForSequenceClassification, AdamW
path='../input/bert-base-uncased'
tokenizer= BertTokenizer.from_pretrained(path)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
train = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test=pd.read_csv('../input/commonlitreadabilityprize/test.csv')

# Lets start defining our reading parameters:
## 1) Sentence Length
## 2) Number of Dale words
## 3) Number of hard words defined by phenomes
## 4) Spacy parameter where we look at 'CONJ', 'CCONJ', 'VERB' in a sentence and also if its a sentence starting with a pronoun.

In [ ]:
#sent length by number of sents

def sent_len(text):
    text2=re.sub(r'[^\w\s]','', text)
    doc= nlp(text2)
    sents= doc.sents
    all_len=[]
    for sent in sents:
        words= len(sent.text.split())
        all_len.append(words)
    return sum(all_len)/len(all_len)



#number of easy dale words

dale_chall_words="a|able|aboard|about|above|absent|accept|accident|account|ache|aching|acorn|acre|across|act|acts|add|address|admire|adventure|afar|afraid|after|afternoon|afterward|afterwards|again|against|age|aged|ago|agree|ah|ahead|aid|aim|air|airfield|airplane|airport|airship|airy|alarm|alike|alive|all|alley|alligator|allow|almost|alone|along|aloud|already|also|always|am|America|American|among|amount|an|and|angel|anger|angry|animal|another|answer|ant|any|anybody|anyhow|anyone|anything|anyway|anywhere|apart|apartment|ape|apiece|appear|apple|April|apron|are|aren't|arise|arithmetic|arm|armful|army|arose|around|arrange|arrive|arrived|arrow|art|artist|as|ash|ashes|aside|ask|asleep|at|ate|attack|attend|attention|August|aunt|author|auto|automobile|autumn|avenue|awake|awaken|away|awful|awfully|awhile|ax|axe|baa|babe|babies|back|background|backward|backwards|bacon|bad|badge|badly|bag|bake|baker|bakery|baking|ball|balloon|banana|band|bandage|bang|banjo|bank|banker|bar|barber|bare|barefoot|barely|bark|barn|barrel|base|baseball|basement|basket|bat|batch|bath|bathe|bathing|bathroom|bathtub|battle|battleship|bay|be|beach|bead|beam|bean|bear|beard|beast|beat|beating|beautiful|beautify|beauty|became|because|become|becoming|bed|bedbug|bedroom|bedspread|bedtime|bee|beech|beef|beefsteak|beehive|been|beer|beet|before|beg|began|beggar|begged|begin|beginning|begun|behave|behind|being|believe|bell|belong|below|belt|bench|bend|beneath|bent|berries|berry|beside|besides|best|bet|better|between|bib|bible|bicycle|bid|big|bigger|bill|billboard|bin|bind|bird|birth|birthday|biscuit|bit|bite|biting|bitter|black|blackberry|blackbird|blackboard|blackness|blacksmith|blame|blank|blanket|blast|blaze|bleed|bless|blessing|blew|blind|blindfold|blinds|block|blood|bloom|blossom|blot|blow|blue|blueberry|bluebird|blush|board|boast|boat|bob|bobwhite|bodies|body|boil|boiler|bold|bone|bonnet|boo|book|bookcase|bookkeeper|boom|boot|born|borrow|boss|both|bother|bottle|bottom|bought|bounce|bow|bowl|bow-wow|box|boxcar|boxer|boxes|boy|boyhood|bracelet|brain|brake|bran|branch|brass|brave|bread|break|breakfast|breast|breath|breathe|breeze|brick|bride|bridge|bright|brightness|bring|broad|broadcast|broke|broken|brook|broom|brother|brought|brown|brush|bubble|bucket|buckle|bud|buffalo|bug|buggy|build|building|built|bulb|bull|bullet|bum|bumblebee|bump|bun|bunch|bundle|bunny|burn|burst|bury|bus|bush|bushel|business|busy|but|butcher|butt|butter|buttercup|butterfly|buttermilk|butterscotch|button|buttonhole|buy|buzz|by|bye|cab|cabbage|cabin|cabinet|cackle|cage|cake|calendar|calf|call|caller|calling|came|camel|camp|campfire|can|canal|canary|candle|candlestick|candy|cane|cannon|cannot|canoe|can't|canyon|cap|cape|capital|captain|car|card|cardboard|care|careful|careless|carelessness|carload|carpenter|carpet|carriage|carrot|carry|cart|carve|case|cash|cashier|castle|cat|catbird|catch|catcher|caterpillar|catfish|catsup|cattle|caught|cause|cave|ceiling|cell|cellar|cent|center|cereal|certain|certainly|chain|chair|chalk|champion|chance|change|chap|charge|charm|chart|chase|chatter|cheap|cheat|check|checkers|cheek|cheer|cheese|cherry|chest|chew|chick|chicken|chief|child|childhood|children|chill|chilly|chimney|chin|china|chip|chipmunk|chocolate|choice|choose|chop|chorus|chose|chosen|christen|Christmas|church|churn|cigarette|circle|circus|citizen|city|clang|clap|class|classmate|classroom|claw|clay|clean|cleaner|clear|clerk|clever|click|cliff|climb|clip|cloak|clock|close|closet|cloth|clothes|clothing|cloud|cloudy|clover|clown|club|cluck|clump|coach|coal|coast|coat|cob|cobbler|cocoa|coconut|cocoon|cod|codfish|coffee|coffeepot|coin|cold|collar|college|color|colored|colt|column|comb|come|comfort|comic|coming|company|compare|conductor|cone|connect|coo|cook|cooked|cooking|cookie|cookies|cool|cooler|coop|copper|copy|cord|cork|corn|corner|correct|cost|cot|cottage|cotton|couch|cough|could|couldn't|count|counter|country|county|course|court|cousin|cover|cow|coward|cowardly|cowboy|cozy|crab|crack|cracker|cradle|cramps|cranberry|crank|cranky|crash|crawl|crazy|cream|creamy|creek|creep|crept|cried|croak|crook|crooked|crop|cross|crossing|cross-eyed|crow|crowd|crowded|crown|cruel|crumb|crumble|crush|crust|cry|cries|cub|cuff|cup|cuff|cup|cupboard|cupful|cure|curl|curly|curtain|curve|cushion|custard|customer|cut|cute|cutting|dab|dad|daddy|daily|dairy|daisy|dam|damage|dame|damp|dance|dancer|dancing|dandy|danger|dangerous|dare|dark|darkness|darling|darn|dart|dash|date|daughter|dawn|day|daybreak|daytime|dead|deaf|deal|dear|death|December|decide|deck|deed|deep|deer|defeat|defend|defense|delight|den|dentist|depend|deposit|describe|desert|deserve|desire|desk|destroy|devil|dew|diamond|did|didn't|die|died|dies|difference|different|dig|dim|dime|dine|ding-dong|dinner|dip|direct|direction|dirt|dirty|discover|dish|dislike|dismiss|ditch|dive|diver|divide|do|dock|doctor|does|doesn't|dog|doll|dollar|dolly|done|donkey|don't|door|doorbell|doorknob|doorstep|dope|dot|double|dough|dove|down|downstairs|downtown|dozen|drag|drain|drank|draw|drawer|draw|drawing|dream|dress|dresser|dressmaker|drew|dried|drift|drill|drink|drip|drive|driven|driver|drop|drove|drown|drowsy|drub|drum|drunk|dry|duck|due|dug|dull|dumb|dump|during|dust|dusty|duty|dwarf|dwell|dwelt|dying|each|eager|eagle|ear|early|earn|earth|east|eastern|easy|eat|eaten|edge|egg|eh|eight|eighteen|eighth|eighty|either|elbow|elder|eldest|electric|electricity|elephant|eleven|elf|elm|else|elsewhere|empty|end|ending|enemy|engine|engineer|English|enjoy|enough|enter|envelope|equal|erase|eraser|errand|escape|eve|even|evening|ever|every|everybody|everyday|everyone|everything|everywhere|evil|exact|except|exchange|excited|exciting|excuse|exit|expect|explain|extra|eye|eyebrow|fable|face|facing|fact|factory|fail|faint|fair|fairy|faith|fake|fall|false|family|fan|fancy|far|faraway|fare|farmer|farm|farming|far-off|farther|fashion|fast|fasten|fat|father|fault|favor|favorite|fear|feast|feather|February|fed|feed|feel|feet|fell|fellow|felt|fence|fever|few|fib|fiddle|field|fife|fifteen|fifth|fifty|fig|fight|figure|file|fill|film|finally|find|fine|finger|finish|fire|firearm|firecracker|fireplace|fireworks|firing|first|fish|fisherman|fist|fit|fits|five|fix|flag|flake|flame|flap|flash|flashlight|flat|flea|flesh|flew|flies|flight|flip|flip-flop|float|flock|flood|floor|flop|flour|flow|flower|flowery|flutter|fly|foam|fog|foggy|fold|folks|follow|following|fond|food|fool|foolish|foot|football|footprint|for|forehead|forest|forget|forgive|forgot|forgotten|fork|form|fort|forth|fortune|forty|forward|fought|found|fountain|four|fourteen|fourth|fox|frame|free|freedom|freeze|freight|French|fresh|fret|Friday|fried|friend|friendly|friendship|frighten|frog|from|front|frost|frown|froze|fruit|fry|fudge|fuel|full|fully|fun|funny|fur|furniture|further|fuzzy|gain|gallon|gallop|game|gang|garage|garbage|garden|gas|gasoline|gate|gather|gave|gay|gear|geese|general|gentle|gentleman|gentlemen|geography|get|getting|giant|gift|gingerbread|girl|give|given|giving|glad|gladly|glance|glass|glasses|gleam|glide|glory|glove|glow|glue|go|going|goes|goal|goat|gobble|God|god|godmother|gold|golden|goldfish|golf|gone|good|goods|goodbye|good-by|goodbye|good-bye|good-looking|goodness|goody|goose|gooseberry|got|govern|government|gown|grab|gracious|grade|grain|grand|grandchild|grandchildren|granddaughter|grandfather|grandma|grandmother|grandpa|grandson|grandstand|grape|grapes|grapefruit|grass|grasshopper|grateful|grave|gravel|graveyard|gravy|gray|graze|grease|great|green|greet|grew|grind|groan|grocery|ground|group|grove|grow|guard|guess|guest|guide|gulf|gum|gun|gunpowder|guy|ha|habit|had|hadn't|hail|hair|haircut|hairpin|half|hall|halt|ham|hammer|hand|handful|handkerchief|handle|handwriting|hang|happen|happily|happiness|happy|harbor|hard|hardly|hardship|hardware|hare|hark|harm|harness|harp|harvest|has|hasn't|haste|hasten|hasty|hat|hatch|hatchet|hate|haul|have|haven't|having|hawk|hay|hayfield|haystack|he|head|headache|heal|health|healthy|heap|hear|hearing|heard|heart|heat|heater|heaven|heavy|he'd|heel|height|held|hell|he'll|hello|helmet|help|helper|helpful|hem|hen|henhouse|her|hers|herd|here|here's|hero|herself|he's|hey|hickory|hid|hidden|hide|high|highway|hill|hillside|hilltop|hilly|him|himself|hind|hint|hip|hire|his|hiss|history|hit|hitch|hive|ho|hoe|hog|hold|holder|hole|holiday|hollow|holy|home|homely|homesick|honest|honey|honeybee|honeymoon|honk|honor|hood|hoof|hook|hoop|hop|hope|hopeful|hopeless|horn|horse|horseback|horseshoe|hose|hospital|host|hot|hotel|hound|hour|house|housetop|housewife|housework|how|however|howl|hug|huge|hum|humble|hump|hundred|hung|hunger|hungry|hunk|hunt|hunter|hurrah|hurried|hurry|hurt|husband|hush|hut|hymn|I|ice|icy|I'd|idea|ideal|if|ill|I'll|I'm|important|impossible|improve|in|inch|inches|income|indeed|Indian|indoors|ink|inn|insect|inside|instant|instead|insult|intend|interested|interesting|into|invite|iron|is|island|isn't|it|its|it's|itself|I've|ivory|ivy|jacket|jacks|jail|jam|January|jar|jaw|jay|jelly|jellyfish|jerk|jig|job|jockey|join|joke|joking|jolly|journey|joy|joyful|joyous|judge|jug|juice|juicy|July|jump|June|junior|junk|just|keen|keep|kept|kettle|key|kick|kid|kill|killed|kind|kindly|kindness|king|kingdom|kiss|kitchen|kite|kitten|kitty|knee|kneel|knew|knife|knit|knives|knob|knock|knot|know|known|lace|lad|ladder|ladies|lady|laid|lake|lamb|lame|lamp|land|lane|language|lantern|lap|lard|large|lash|lass|last|late|laugh|laundry|law|lawn|lawyer|lay|lazy|lead|leader|leaf|leak|lean|leap|learn|learned|least|leather|leave|leaving|led|left|leg|lemon|lemonade|lend|length|less|lesson|let|let's|letter|letting|lettuce|level|liberty|library|lice|lick|lid|lie|life|lift|light|lightness|lightning|like|likely|liking|lily|limb|lime|limp|line|linen|lion|lip|list|listen|lit|little|live|lives|lively|liver|living|lizard|load|loaf|loan|loaves|lock|locomotive|log|lone|lonely|lonesome|long|look|lookout|loop|loose|lord|lose|loser|loss|lost|lot|loud|love|lovely|lover|low|luck|lucky|lumber|lump|lunch|lying|ma|machine|machinery|mad|made|magazine|magic|maid|mail|mailbox|mailman|major|make|making|male|mama|mamma|man|manager|mane|manger|many|map|maple|marble|march|March|mare|mark|market|marriage|married|marry|mask|mast|master|mat|match|matter|mattress|may|May|maybe|mayor|maypole|me|meadow|meal|mean|means|meant|measure|meat|medicine|meet|meeting|melt|member|men|mend|meow|merry|mess|message|met|metal|mew|mice|middle|midnight|might|mighty|mile|milk|milkman|mill|miler|million|mind|mine|miner|mint|minute|mirror|mischief|miss|Miss|misspell|mistake|misty|mitt|mitten|mix|moment|Monday|money|monkey|month|moo|moon|moonlight|moose|mop|more|morning|morrow|moss|most|mostly|mother|motor|mount|mountain|mouse|mouth|move|movie|movies|moving|mow|Mr.|Mrs.|much|mud|muddy|mug|mule|multiply|murder|music|must|my|myself|nail|name|nap|napkin|narrow|nasty|naughty|navy|near|nearby|nearly|neat|neck|necktie|need|needle|needn't|Negro|neighbor|neighborhood|neither|nerve|nest|net|never|nevermore|new|news|newspaper|next|nibble|nice|nickel|night|nightgown|nine|nineteen|ninety|no|nobody|nod|noise|noisy|none|noon|nor|north|northern|nose|not|note|nothing|notice|November|now|nowhere|number|nurse|nut|oak|oar|oatmeal|oats|obey|ocean|o'clock|October|odd|of|off|offer|office|officer|often|oh|oil|old|old-fashioned|on|once|one|onion|only|onward|open|or|orange|orchard|order|ore|organ|other|otherwise|ouch|ought|our|ours|ourselves|out|outdoors|outfit|outlaw|outline|outside|outward|oven|over|overalls|overcoat|overeat|overhead|overhear|overnight|overturn|owe|owing|owl|own|owner|ox|pa|pace|pack|package|pad|page|paid|pail|pain|painful|paint|painter|painting|pair|pal|palace|pale|pan|pancake|pane|pansy|pants|papa|paper|parade|pardon|parent|park|part|partly|partner|party|pass|passenger|past|paste|pasture|pat|patch|path|patter|pave|pavement|paw|pay|payment|pea|peas|peace|peaceful|peach|peaches|peak|peanut|pear|pearl|peck|peek|peel|peep|peg|pen|pencil|penny|people|pepper|peppermint|perfume|perhaps|person|pet|phone|piano|pick|pickle|picnic|picture|pie|piece|pig|pigeon|piggy|pile|pill|pillow|pin|pine|pineapple|pink|pint|pipe|pistol|pit|pitch|pitcher|pity|place|plain|plan|plane|plant|plate|platform|platter|play|player|playground|playhouse|playmate|plaything|pleasant|please|pleasure|plenty|plow|plug|plum|pocket|pocketbook|poem|point|poison|poke|pole|police|policeman|polish|polite|pond|ponies|pony|pool|poor|pop|popcorn|popped|porch|pork|possible|post|postage|postman|pot|potato|potatoes|pound|pour|powder|power|powerful|praise|pray|prayer|prepare|present|pretty|price|prick|prince|princess|print|prison|prize|promise|proper|protect|proud|prove|prune|public|puddle|puff|pull|pump|pumpkin|punch|punish|pup|pupil|puppy|pure|purple|purse|push|puss|pussy|pussycat|put|putting|puzzle|quack|quart|quarter|queen|queer|question|quick|quickly|quiet|quilt|quit|quite|rabbit|race|rack|radio|radish|rag|rail|railroad|railway|rain|rainy|rainbow|raise|raisin|rake|ram|ran|ranch|rang|rap|rapidly|rat|rate|rather|rattle|raw|ray|reach|read|reader|reading|ready|real|really|reap|rear|reason|rebuild|receive|recess|record|red|redbird|redbreast|refuse|reindeer|rejoice|remain|remember|remind|remove|rent|repair|repay|repeat|report|rest|return|review|reward|rib|ribbon|rice|rich|rid|riddle|ride|rider|riding|right|rim|ring|rip|ripe|rise|rising|river|road|roadside|roar|roast|rob|robber|robe|robin|rock|rocky|rocket|rode|roll|roller|roof|room|rooster|root|rope|rose|rosebud|rot|rotten|rough|round|route|row|rowboat|royal|rub|rubbed|rubber|rubbish|rug|rule|ruler|rumble|run|rung|runner|running|rush|rust|rusty|rye|sack|sad|saddle|sadness|safe|safety|said|sail|sailboat|sailor|saint|salad|sale|salt|same|sand|sandy|sandwich|sang|sank|sap|sash|sat|satin|satisfactory|Saturday|sausage|savage|save|savings|saw|say|scab|scales|scare|scarf|school|schoolboy|schoolhouse|schoolmaster|schoolroom|scorch|score|scrap|scrape|scratch|scream|screen|screw|scrub|sea|seal|seam|search|season|seat|second|secret|see|seeing|seed|seek|seem|seen|seesaw|select|self|selfish|sell|send|sense|sent|sentence|separate|September|servant|serve|service|set|setting|settle|settlement|seven|seventeen|seventh|seventy|several|sew|shade|shadow|shady|shake|shaker|shaking|shall|shame|shan't|shape|share|sharp|shave|she|she'd|she'll|she's|shear|shears|shed|sheep|sheet|shelf|shell|shepherd|shine|shining|shiny|ship|shirt|shock|shoe|shoemaker|shone|shook|shoot|shop|shopping|shore|short|shot|should|shoulder|shouldn't|shout|shovel|show|shower|shut|shy|sick|sickness|side|sidewalk|sideways|sigh|sight|sign|silence|silent|silk|sill|silly|silver|simple|sin|since|sing|singer|single|sink|sip|sir|sis|sissy|sister|sit|sitting|six|sixteen|sixth|sixty|size|skate|skater|ski|skin|skip|skirt|sky|slam|slap|slate|slave|sled|sleep|sleepy|sleeve|sleigh|slept|slice|slid|slide|sling|slip|slipped|slipper|slippery|slit|slow|slowly|sly|smack|small|smart|smell|smile|smoke|smooth|snail|snake|snap|snapping|sneeze|snow|snowy|snowball|snowflake|snuff|snug|so|soak|soap|sob|socks|sod|soda|sofa|soft|soil|sold|soldier|sole|some|somebody|somehow|someone|something|sometime|sometimes|somewhere|son|song|soon|sore|sorrow|sorry|sort|soul|sound|soup|sour|south|southern|space|spade|spank|sparrow|speak|speaker|spear|speech|speed|spell|spelling|spend|spent|spider|spike|spill|spin|spinach|spirit|spit|splash|spoil|spoke|spook|spoon|sport|spot|spread|spring|springtime|sprinkle|square|squash|squeak|squeeze|squirrel|stable|stack|stage|stair|stall|stamp|stand|star|stare|start|starve|state|station|stay|steak|steal|steam|steamboat|steamer|steel|steep|steeple|steer|stem|step|stepping|stick|sticky|stiff|still|stillness|sting|stir|stitch|stock|stocking|stole|stone|stood|stool|stoop|stop|stopped|stopping|store|stork|stories|storm|stormy|story|stove|straight|strange|stranger|strap|straw|strawberry|stream|street|stretch|string|strip|stripes|strong|stuck|study|stuff|stump|stung|subject|such|suck|sudden|suffer|sugar|suit|sum|summer|sun|Sunday|sunflower|sung|sunk|sunlight|sunny|sunrise|sunset|sunshine|supper|suppose|sure|surely|surface|surprise|swallow|swam|swamp|swan|swat|swear|sweat|sweater|sweep|sweet|sweetness|sweetheart|swell|swept|swift|swim|swimming|swing|switch|sword|swore|table|tablecloth|tablespoon|tablet|tack|tag|tail|tailor|take|taken|taking|tale|talk|talker|tall|tame|tan|tank|tap|tape|tar|tardy|task|taste|taught|tax|tea|teach|teacher|team|tear|tease|teaspoon|teeth|telephone|tell|temper|ten|tennis|tent|term|terrible|test|than|thank|thanks|thankful|Thanksgiving|that|that's|the|theater|thee|their|them|then|there|these|they|they'd|they'll|they're|they've|thick|thief|thimble|thin|thing|think|third|thirsty|thirteen|thirty|this|thorn|those|though|thought|thousand|thread|three|threw|throat|throne|through|throw|thrown|thumb|thunder|Thursday|thy|tick|ticket|tickle|tie|tiger|tight|till|time|tin|tinkle|tiny|tip|tiptoe|tire|tired|title|to|toad|toadstool|toast|tobacco|today|toe|together|toilet|told|tomato|tomorrow|ton|tone|tongue|tonight|too|took|tool|toot|tooth|toothbrush|toothpick|top|tore|torn|toss|touch|tow|toward|towards|towel|tower|town|toy|trace|track|trade|train|tramp|trap|tray|treasure|treat|tree|trick|tricycle|tried|trim|trip|trolley|trouble|truck|true|truly|trunk|trust|truth|try|tub|Tuesday|tug|tulip|tumble|tune|tunnel|turkey|turn|turtle|twelve|twenty|twice|twig|twin|two|ugly|umbrella|uncle|under|understand|underwear|undress|unfair|unfinished|unfold|unfriendly|unhappy|unhurt|uniform|United States|unkind|unknown|unless|unpleasant|until|unwilling|up|upon|upper|upset|upside|upstairs|uptown|upward|us|use|used|useful|valentine|valley|valuable|value|vase|vegetable|velvet|very|vessel|victory|view|village|vine|violet|visit|visitor|voice|vote|wag|wagon|waist|wait|wake|waken|walk|wall|walnut|want|war|warm|warn|was|wash|washer|washtub|wasn't|waste|watch|watchman|water|watermelon|waterproof|wave|wax|way|wayside|we|weak|weakness|weaken|wealth|weapon|wear|weary|weather|weave|web|we'd|wedding|Wednesday|wee|weed|week|we'll|weep|weigh|welcome|well|went|were|we're|west|western|wet|we've|whale|what|what's|wheat|wheel|when|whenever|where|which|while|whip|whipped|whirl|whisky|whiskey|whisper|whistle|white|who|who'd|whole|who'll|whom|who's|whose|why|wicked|wide|wife|wiggle|wild|wildcat|will|willing|willow|win|wind|windy|windmill|window|wine|wing|wink|winner|winter|wipe|wire|wise|wish|wit|witch|with|without|woke|wolf|woman|women|won|wonder|wonderful|won't|wood|wooden|woodpecker|woods|wool|woolen|word|wore|work|worker|workman|world|worm|worn|worry|worse|worst|worth|would|wouldn't|wound|wove|wrap|wrapped|wreck|wren|wring|write|writing|written|wrong|wrote|wrung|yard|yarn|year|yell|yellow|yes|yesterday|yet|yolk|yonder|you|you'd|you'll|young|youngster|your|yours|you're|yourself|yourselves|youth|you've"
dale_chall_list=dale_chall_words.split('|')


#number of hardwords according to phenomes
from nltk.corpus import cmudict
d = cmudict.dict()

def nsyl(word):
    try:
        return [len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]]
    except KeyError:
        #if word not found in cmudict
        return syllables(word)

def syllables(word):
    #referred from stackoverflow.com/questions/14541303/count-the-number-of-syllables-in-a-word
    count = 0
    vowels = 'aeiouy'
    word = word.lower()
    if word[0] in vowels:
        count +=1
    for index in range(1,len(word)):
        if word[index] in vowels and word[index-1] not in vowels:
            count +=1
    if word.endswith('e'):
        count -= 1
    if word.endswith('le'):
        count += 1
    if count == 0:
        count += 1
    return count

def hard_word(text):
    doc= nlp(text)
    sents= doc.sents
    total_score=[]
    for sent in sents:
        #print(sent)
    # Initiating hard and total words 
        hard_words=0
        count=0
        
        text2=re.sub(r'[^\w\s]','', sent.text) 
        for i in text2.split(' ') :
            try:
                count+=1
                ns=nsyl(i.lower())
                if type(ns)==int :
                    if ns>2 :
                        hard_words+=1
                else:
                    if ns[0]>2 :
                        hard_words+=1
            except:
                pass
        score= hard_words/count
        total_score.append(score)
    return sum(total_score)/len(total_score)



#scores based on the occurance of 'CONJ', 'CCONJ', 'VERB' in the sentence. Also extended sentences with PRON. 

def spacy_trial(text):
    try:
        sentences= text.split('.')
        all_scores=[]
        for num,sent in enumerate(sentences):
            
            sent=nlp(sent)
            if len(sent)>0:
                definers=[i for i in sent if i.pos_ in ['CONJ', 'CCONJ', 'VERB']]
                definers2=[i for num,i in enumerate(sent) if i.pos_=="PRON" and sent[num].is_title==False]
                
                [definers.append(i) for i in definers2]
                
                score= len(definers)/len(sent.text.split())
                all_scores.append(score)
                
        return sum(all_scores)/len(all_scores)
    except:
        return 0
        

# Apply the paramenetrs on training dataset

In [ ]:
# =============================================================================
# Apply on train
# =============================================================================

train['avg_sent_len']= 0
train['dale_words']= 0
train['hard_word']= 0
train['spacy_trial']= 0

def apply_reading(text):
    avg_sent_len= sent_len(text)
    dale_word= len([i for i in text.split() if i not in dale_chall_list])
    hardword= hard_word(text)
    spacytrial= spacy_trial(text)
    
    return avg_sent_len,dale_word,hardword,spacytrial

for i in range(0, len(train)):
    print(".", end="", flush=True)
    train['avg_sent_len'].iloc[i],train['dale_words'].iloc[i],train['hard_word'].iloc[i],train['spacy_trial'].iloc[i]= apply_reading(train[ 'excerpt'].iloc[i])
    


# Scale the new variables

In [ ]:
train['avg_sent_len']=[(i-min(train['avg_sent_len']))/(max(train['avg_sent_len'])-min(train['avg_sent_len'])) for i in train['avg_sent_len']]
train['dale_words']=[(i-min(train['dale_words']))/(max(train['dale_words'])-min(train['dale_words'])) for i in train['dale_words']]
train['hard_word']=[(i-min(train['hard_word']))/(max(train['hard_word'])-min(train['hard_word'])) for i in train['hard_word']]
train['spacy_trial']=[(i-min(train['spacy_trial']))/(max(train['spacy_trial'])-min(train['spacy_trial'])) for i in train['spacy_trial']]


# Create a Dataloader

In [ ]:
# =============================================================================
# data loader
# =============================================================================

MAX_LEN=512
#data loader
class loader(torch.utils.data.Dataset):
    def __init__(self, df, test=True):
        super().__init__()

        self.df = df        
        self.test = test
        self.text = df.excerpt.tolist()
        self.avg_sent_len= df.avg_sent_len.tolist()
        self.dale_words= df.dale_words.tolist()
        self.hard_word= df.hard_word.tolist()
        self.spacy_trial= df.spacy_trial.tolist()
        #self.text = [text.replace("\n", " ") for text in self.text]
        
        if not self.test:
            self.target = torch.tensor(df.target.values, dtype=torch.float32)
    
        self.encoded = tokenizer.batch_encode_plus(
            self.text,
            padding = 'max_length',            
            max_length = MAX_LEN,
            truncation = True,
            return_attention_mask=True
        )        
 

    def __len__(self):
        return len(self.df)

    
    def __getitem__(self, index):        
        input_ids = torch.tensor(self.encoded['input_ids'][index])
        attention_mask = torch.tensor(self.encoded['attention_mask'][index])
        avg_sent_len=torch.tensor(self.avg_sent_len[index])
        dale_words=torch.tensor(self.dale_words[index])
        hard_word=torch.tensor(self.hard_word[index])
        spacy_trial=torch.tensor(self.spacy_trial[index])
        
        if self.test:
            return (input_ids, attention_mask,avg_sent_len,dale_words,hard_word,spacy_trial)            
        else:
            target = self.target[index]
            return (input_ids, attention_mask, avg_sent_len,dale_words,hard_word,spacy_trial,target)


## Define the model
# we will start from a bert sequence classification regression model and concatenate the new variables

In [ ]:
class Model_bs(nn.Module):
    def __init__(self):
        super().__init__()

        config = BertConfig.from_pretrained(path)
        # config.update({"output_hidden_states":True, 
        #                "hidden_dropout_prob": 0.0,
        #                "layer_norm_eps": 1e-7})                       
        
        self.bs=BertForSequenceClassification.from_pretrained(path, num_labels=1)
            
        self.reading_params = nn.Sequential(            
            nn.Linear(4, 32), 
            #nn.BatchNorm1d(32),
            nn.ReLU(),                       
            nn.Linear(32, 1),
            
        )        
        self.fc1= nn.Sequential(nn.Linear(1,10))
        
        
        self.relu =  nn.ReLU()
        self.tanh =  nn.Tanh()
        self.dropout = nn.Dropout(0.1)
        self.bn11 = nn.BatchNorm1d(14)
        self.regressor = nn.Sequential(                        
            nn.Linear(11, 1)                        
        )
        

    def forward(self, input_ids, attention_mask,avg_sent_len,dale_words,hard_word,spacy_trial):
        bs_output = self.bs(input_ids=input_ids,attention_mask=attention_mask)        
        bs_output=bs_output['logits']
        out= self.fc1(bs_output)
        
        
        x=torch.cat((avg_sent_len.view(avg_sent_len.shape[0],1),dale_words.view(dale_words.shape[0],1)),1)
        
        x=torch.cat((x,hard_word.view(hard_word.shape[0],1)),1)
        
        x=torch.cat((x,spacy_trial.view(spacy_trial.shape[0],1)),1)
        x= self.reading_params(x)
        
        out= torch.cat((out,x),1)
        #out= self.relu(out)
        
        
        
        # Now we reduce the context vector to the prediction score.
        return self.regressor(out)


# Define the train  and evaluate function

In [ ]:
# =============================================================================
# train function
# =============================================================================

def train_model():
  
  model.train()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save model predictions
  total_preds=[]
  
  # iterate over batches
  for step, batch in enumerate(train_loader):
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_loader)))
      
    # push the batch to gpu
    batch = [r.to(device) for r in batch]
    
    input_ids, attention_mask, avg_sent_len,dale_words,hard_word,spacy_trial,target = batch
    # model = Model_bs().to(device)
    # optimizer = AdamW(model.parameters(),lr = .00005)  

    
    # clear previously calculated gradients 
    model.zero_grad()
    
    # get model predictions for the current batch
    preds = model(input_ids, attention_mask, avg_sent_len,dale_words,hard_word,spacy_trial)
    
    # compute the loss between actual and predicted values
    mseloss = nn.MSELoss()
    loss=mseloss(preds.view(-1), target)
    
    
    # add on to the total loss
    total_loss = total_loss + loss.item()
    
    # backward pass to calculate the gradients
    loss.backward()
    
    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    
    # update parameters
    optimizer.step()
    
    # model predictions are stored on GPU. So, push it to CPU
    preds=preds.detach().cpu().numpy()
    
    # append the model predictions
    total_preds.append(preds)
    
  # compute the training loss of the epoch
  avg_loss = total_loss / len(train_loader)
  
  # predictions are in the form of (no. of batches, size of batch, no. of classes).
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  #returns the loss and predictions
  return avg_loss, total_preds


# =============================================================================
# evaluate function
# =============================================================================
def evaluate():
  
  print("\nEvaluating...")
  
  # deactivate dropout layers
  model.eval()

  total_loss = 0
  
  # empty list to save the model predictions
  total_preds = []

  # iterate over batches
  for step, batch in enumerate(val_loader):
      #print(step)
      #break
    
    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0:
      
      # Calculate elapsed time in minutes.
      #elapsed = format_time(time.time() - t0)
            
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_loader)))

    # push the batch to gpu
    batch = [t.to(device) for t in batch]

    input_ids, attention_mask, avg_sent_len,dale_words,hard_word,spacy_trial,target = batch

    # deactivate autograd
    with torch.no_grad():
      
      # model predictions
      preds = model(input_ids, attention_mask, avg_sent_len,dale_words,hard_word,spacy_trial)
      mseloss = nn.MSELoss()
      loss=mseloss(preds.view(-1), target)
      # compute the validation loss between actual and predicted values
      #loss = cross_entropy(preds,labels)

      total_loss = total_loss + loss.item()

      preds = preds.detach().cpu().numpy()

      total_preds.append(preds)

  # compute the validation loss of the epoch
  avg_loss = total_loss / len(val_loader) 

  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  return avg_loss, total_preds



# split and call the data loaders

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( train, train['target'], test_size=0.2, random_state=4, shuffle=True)

train_df= X_train
dataset=loader(train_df, test=False)
train_loader=torch.utils.data.DataLoader(dataset, batch_size=4,drop_last=True, shuffle=True)

val_df= X_test
dataset=loader(val_df, test=False)
val_loader=torch.utils.data.DataLoader(dataset, batch_size=4,drop_last=True, shuffle=False)



# INitialize the model

In [ ]:
model = Model_bs().to(device)
optimizer = AdamW(model.parameters(),lr = .00005)  


# Treain the model, I have taken ten epochs and saved the best model on validation loss

In [ ]:
epochs=10
best_valid_loss = float('inf')
train_losses=[]
valid_losses=[]
models_saved=[]

for epoch in range(epochs):
 
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))

    
    train_loss, _ = train_model()

    
    valid_loss, _ = evaluate()
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '/kaggle/working/model_saved.pt')
        models_saved.append(epoch)
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')



# load the best model and train on the validation data

In [ ]:
# =============================================================================
# load best model and train further on the 20 pc data left for training
# =============================================================================

#load weights of best model
path = 'final_model/model_saved.pt'
model.load_state_dict(torch.load(path))


del train_loader
train_loader= val_loader
for _ in range(0,2):
    train_loss, _ = train_model()
    print(train_loss)


# Create the reading parameters for the test dataset

In [ ]:
#create test
test['avg_sent_len']= 0
test['dale_words']= 0
test['hard_word']= 0
test['spacy_trial']= 0

def apply_reading(text):
    avg_sent_len= sent_len(text)
    dale_word= len([i for i in text.split() if i not in dale_chall_list])
    hardword= hard_word(text)
    spacytrial= spacy_trial(text)
    
    return avg_sent_len,dale_word,hardword,spacytrial

for i in range(0, len(test)):
    print(".", end="", flush=True)
    test['avg_sent_len'].iloc[i],test['dale_words'].iloc[i],test['hard_word'].iloc[i],test['spacy_trial'].iloc[i]= apply_reading(test[ 'excerpt'].iloc[i])
    

# apply scaling

In [ ]:
test['avg_sent_len']=[(i-min(test['avg_sent_len']))/(max(test['avg_sent_len'])-min(test['avg_sent_len'])) for i in test['avg_sent_len']]
test['dale_words']=[(i-min(test['dale_words']))/(max(test['dale_words'])-min(test['dale_words'])) for i in test['dale_words']]
test['hard_word']=[(i-min(test['hard_word']))/(max(test['hard_word'])-min(test['hard_word'])) for i in test['hard_word']]
test['spacy_trial']=[(i-min(test['spacy_trial']))/(max(test['spacy_trial'])-min(test['spacy_trial'])) for i in test['spacy_trial']]

# data loader

In [ ]:
#val_df= train.loc[val_indices]
test_dataset=loader(test, test=True)
test_loader=torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)


# finaly the predict function

In [ ]:
def predict():
  
  print("\npredicting...")
  
  # deactivate dropout layers
  
  total_preds=[]# iterate over batches
  for step,batch in enumerate(test_loader):
    print(".", end="", flush=True)
    # push the batch to gpu
    batch = [t.to(device) for t in batch]
    
    input_ids, attention_mask, avg_sent_len,dale_words,hard_word,spacy_trial = batch
    
    # deactivate autograd
    with torch.no_grad():
        
        # model predictions
        preds = model(input_ids, attention_mask, avg_sent_len,dale_words,hard_word,spacy_trial)
        
        preds = preds.detach().cpu().numpy()
        
        [total_preds.append(i[0]) for i in preds]
  return total_preds


preds= predict()
test['target']= preds

# Submit results!

In [ ]:
submission= test[['id', 'target']]
submission.to_csv('/kaggle/working/submission.csv', index=False)